# Testing Data Modelling Techniques

This notebook is a complete mess. I'll make it clean later on (still testing some stuff).

Techniques to be tested:
- User2Vec
- Network Analysis (Tweets)
- Network Analysis (Users)
- Popular vs unpopular users identification
- Topic Modelling
- Text summarization

In [2]:
from elasticsearch import Elasticsearch
import pandas as pd
import numpy as np
import eland as ed
import networkx as nx

# uncomment this if database is not already open (and give ES a couple minutes to set up)
#!make database

es = Elasticsearch(['localhost'])

In [3]:
# drop empty coluns
ed_df = ed.read_es('localhost', 'twitter')

df = ed_df[
    ['tweet_id', 'user_id', 'name', 'full_text_processed',
    'original_tweet_id_str', 'quoted_status_id_str', 'in_reply_to_status_id_str',
    'is_quote_status', 'is_retweet', 'is_reply', 'is_translator', 'is_original']
].to_pandas().fillna(np.nan)

## Network Analysis (Tweets)
There is possibly little to no information to be taken from this section, but I kept it anyway.

In [16]:
#df_network = ed_df[
#    ['tweet_id', 
#    'original_tweet_id_str', 'quoted_status_id_str', 'in_reply_to_status_id_str',
#    'is_quote_status', 'is_retweet', 'is_reply', 'is_translator', 'is_original']
#].to_pandas().fillna(np.nan).rename(columns={'tweet_id': 'target'})

In [17]:
# remove the annoying scientific notation from id columns
pd.set_option('display.float_format', lambda x: '%.0f' % x)

def get_source(row):
    if row['is_quote_status']:
        return row['quoted_status_id_str']
    elif row['is_retweet']:
        return row['original_tweet_id_str']
    elif row['is_reply']:
        return row['in_reply_to_status_id_str']
    else:
        return np.nan

df_network['source'] = df_network.apply(get_source, axis=1)

In [20]:
df_network['weight'] = 1
df_network[['source', 'target', 'weight']]

,source,target,weight
1263804753286397952,1263801627288521984,1263804753286397952,1
1263820895312732160,1263812617430134785,1263820895312732160,1
1263820892871618560,1263762563566731264,1263820892871618560,1
1263820887897378816,1263788358343499782,1263820887897378816,1
1263820880800595968,1263820878783053056,1263820880800595968,1
...,...,...,...
1263850896217092096,1263848854383849477,1263850896217092096,1
1263850895160348672,1263848854383849477,1263850895160348672,1
1263850832103161856,1263847781094944768,1263850832103161856,1
1263850818689740800,1263848064592183298,1263850818689740800,1


In [32]:
# create a directed graph
G_dir = nx.from_pandas_edgelist(df_network[['source', 'target', 'weight']], create_using=nx.DiGraph())

# create an undirected graph
G = nx.from_pandas_edgelist(df_network[['source', 'target', 'weight']], create_using=nx.Graph())


In [37]:
print(f'Graph basic info: {G_dir.number_of_nodes()} nodes, {G_dir.number_of_edges()} edges.')

Graph basic info: 495233 nodes, 455471 edges.


In [38]:
# compare densities of the two graphs
print(f'Density measurement: \nUndirected: {nx.density(G)}\nDirected: {nx.density(G_dir)}')

Density measurement: 
Undirected: 3.71426127592167e-06
Directed: 1.857130637960835e-06


In [39]:
#out-degree
node_attrs=pd.DataFrame.from_dict(dict(G_dir.out_degree()), 'index').rename(columns={0:'out_degree'})

#in-degree
node_attrs=node_attrs.join(pd.DataFrame.from_dict(dict(G_dir.in_degree()), 'index').rename(columns={0:'in_degree'}))

# weighted out-degree
#node_attrs=node_attrs.join(pd.DataFrame.from_dict(dict(G_dir.out_degree(weight='weight')), 'index').rename(columns={0:'weighted_out_degree'}))

# weighted in-degree
#node_attrs=node_attrs.join(pd.DataFrame.from_dict(dict(G_dir.in_degree(weight='weight')), 'index').rename(columns={0:'weighted_in_degree'}))

# add tweet ids and original info
#node_attrs=node_attrs.join(df.set_index('COMUNE')[["COD_REG","COD_CM", "COD_PRO", "PRO_COM"]])

In [48]:
node_attrs

,out_degree,in_degree
126,1,0
278816194440605696,1,0
401812009810800576,1,0
446602988501491712,1,0
521237811643445184,1,0
...,...,...
nan,1,0
nan,1,0
nan,1,0
nan,1,0


In [ ]:
node_attrs.out_degree.unique()

In [40]:
df_network[['source', 'target', 'weight']]

,source,target,weight
1263804753286397952,1263801627288521984,1263804753286397952,1
1263820895312732160,1263812617430134785,1263820895312732160,1
1263820892871618560,1263762563566731264,1263820892871618560,1
1263820887897378816,1263788358343499782,1263820887897378816,1
1263820880800595968,1263820878783053056,1263820880800595968,1
...,...,...,...
1263850896217092096,1263848854383849477,1263850896217092096,1
1263850895160348672,1263848854383849477,1263850895160348672,1
1263850832103161856,1263847781094944768,1263850832103161856,1
1263850818689740800,1263848064592183298,1263850818689740800,1


## Network Analysis (users)

In [66]:
df_network = ed_df[
    ['tweet_id', 'name',
    'original_tweet_id_str', 'quoted_status_id_str', 'in_reply_to_status_id_str',
    'is_quote_status', 'is_retweet', 'is_reply', 'is_translator', 'is_original']
].to_pandas().fillna(np.nan)

In [61]:
cols = ed_df.columns
cols[cols.str.contains('name')]

#ed_df[[]]

Index(['derived.locations.full_name', 'entities.user_mentions.name',
       'entities.user_mentions.screen_name', 'name', 'screen_name',
       'users_derived_full_name'],
      dtype='object')

## User2Vec
Based on: https://ieeexplore.ieee.org/document/8875952/

How it works:
- Run doc2vec
- Average vector representations for each user


In [4]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess
from gensim.models.word2vec import LineSentence

In [5]:
df_user2vec = df[[
    'tweet_id', 'user_id', 'name', 'full_text_processed', 
    'is_quote_status', 'is_retweet', 'is_reply'
]].copy()

In [50]:
def tokenize(docs, tags=None):
    tags = [None for i in range(len(docs))] \
        if tags is None else tags
    
    for doc, tag in zip(docs, tags):
        tokens = simple_preprocess(doc)
        if tag is None:
            yield tokens
        else:
            yield TaggedDocument(tokens, [tag])

train_corpus = list(tokenize(df_user2vec['full_text_processed'], range(df_user2vec.shape[0])))

In [54]:
model = Doc2Vec(vector_size=10, min_count=2, epochs=40)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

2020-08-12 17:25:46,692 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2020-08-12 17:25:46,942 : INFO : collecting all words and their counts
2020-08-12 17:25:46,947 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-08-12 17:25:47,256 : INFO : PROGRESS: at example #10000, processed 177046 words (577276/s), 5807 word types, 10000 tags
2020-08-12 17:25:47,416 : INFO : PROGRESS: at example #20000, processed 317596 words (891822/s), 8583 word types, 20000 tags
2020-08-12 17:25:47,655 : INFO : PROGRESS: at example #30000, processed 512125 words (821568/s), 11197 word types, 30000 tags
2020-08-12 17:25:47,811 : INFO : PROGRESS: at example #40000, processed 683166 words (1113276/s), 14499 word types, 40000 tags
2020-08-12 17:25:47,998 : INFO : PROGRESS: at example #50000, processed 872601 words (1023998/s), 15765 word types, 50000 tags
2020-08-12 17:25:48,169 : INFO : PROGRESS: at example #60000, processed 1045154 words (1

2020-08-12 17:28:19,794 : INFO : EPOCH 1 - PROGRESS: at 28.13% examples, 169878 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:28:20,837 : INFO : EPOCH 1 - PROGRESS: at 30.63% examples, 170052 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:28:21,983 : INFO : EPOCH 1 - PROGRESS: at 32.15% examples, 162345 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:28:23,020 : INFO : EPOCH 1 - PROGRESS: at 33.68% examples, 158073 words/s, in_qsize 5, out_qsize 0
2020-08-12 17:28:24,142 : INFO : EPOCH 1 - PROGRESS: at 35.14% examples, 152471 words/s, in_qsize 5, out_qsize 0
2020-08-12 17:28:25,157 : INFO : EPOCH 1 - PROGRESS: at 37.10% examples, 150216 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:28:26,281 : INFO : EPOCH 1 - PROGRESS: at 39.68% examples, 150426 words/s, in_qsize 5, out_qsize 0
2020-08-12 17:28:27,353 : INFO : EPOCH 1 - PROGRESS: at 42.65% examples, 152100 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:28:28,391 : INFO : EPOCH 1 - PROGRESS: at 45.42% examples, 152680 words/s, in_qsiz

2020-08-12 17:29:32,687 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-12 17:29:32,688 : INFO : EPOCH - 2 : training on 7755960 raw words (6877843 effective words) took 45.5s, 151277 effective words/s
2020-08-12 17:29:33,709 : INFO : EPOCH 3 - PROGRESS: at 2.39% examples, 162883 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:29:34,730 : INFO : EPOCH 3 - PROGRESS: at 5.39% examples, 181174 words/s, in_qsize 5, out_qsize 0
2020-08-12 17:29:35,745 : INFO : EPOCH 3 - PROGRESS: at 7.75% examples, 181126 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:29:36,759 : INFO : EPOCH 3 - PROGRESS: at 10.62% examples, 184902 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:29:37,788 : INFO : EPOCH 3 - PROGRESS: at 13.10% examples, 182099 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:29:38,804 : INFO : EPOCH 3 - PROGRESS: at 15.66% examples, 180896 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:29:39,853 : INFO : EPOCH 3 - PROGRESS: at 18.59% examples, 184179 words/s, in_qsiz

2020-08-12 17:30:47,255 : INFO : EPOCH 4 - PROGRESS: at 50.47% examples, 135668 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:30:48,277 : INFO : EPOCH 4 - PROGRESS: at 53.42% examples, 137172 words/s, in_qsize 5, out_qsize 0
2020-08-12 17:30:49,325 : INFO : EPOCH 4 - PROGRESS: at 55.39% examples, 137467 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:30:50,326 : INFO : EPOCH 4 - PROGRESS: at 57.65% examples, 138452 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:30:51,394 : INFO : EPOCH 4 - PROGRESS: at 59.92% examples, 137985 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:30:52,439 : INFO : EPOCH 4 - PROGRESS: at 62.41% examples, 138967 words/s, in_qsize 5, out_qsize 0
2020-08-12 17:30:53,450 : INFO : EPOCH 4 - PROGRESS: at 64.72% examples, 139462 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:30:54,450 : INFO : EPOCH 4 - PROGRESS: at 66.53% examples, 139598 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:30:55,454 : INFO : EPOCH 4 - PROGRESS: at 68.79% examples, 140628 words/s, in_qsiz

2020-08-12 17:31:57,316 : INFO : EPOCH 6 - PROGRESS: at 20.01% examples, 198499 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:31:58,366 : INFO : EPOCH 6 - PROGRESS: at 23.35% examples, 198202 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:31:59,379 : INFO : EPOCH 6 - PROGRESS: at 26.63% examples, 200037 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:32:00,395 : INFO : EPOCH 6 - PROGRESS: at 29.71% examples, 201963 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:32:01,405 : INFO : EPOCH 6 - PROGRESS: at 33.01% examples, 202179 words/s, in_qsize 5, out_qsize 0
2020-08-12 17:32:02,424 : INFO : EPOCH 6 - PROGRESS: at 36.21% examples, 201296 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:32:03,467 : INFO : EPOCH 6 - PROGRESS: at 39.53% examples, 201848 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:32:04,532 : INFO : EPOCH 6 - PROGRESS: at 43.02% examples, 202719 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:32:05,609 : INFO : EPOCH 6 - PROGRESS: at 46.38% examples, 201972 words/s, in_qsiz

...

2020-08-12 17:35:23,457 : INFO : EPOCH 12 - PROGRESS: at 50.47% examples, 206677 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:35:24,501 : INFO : EPOCH 12 - PROGRESS: at 53.92% examples, 206688 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:35:25,504 : INFO : EPOCH 12 - PROGRESS: at 56.77% examples, 207147 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:35:26,538 : INFO : EPOCH 12 - PROGRESS: at 60.08% examples, 206773 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:35:27,572 : INFO : EPOCH 12 - PROGRESS: at 63.60% examples, 208125 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:35:28,602 : INFO : EPOCH 12 - PROGRESS: at 66.43% examples, 208157 words/s, in_qsize 5, out_qsize 0
2020-08-12 17:35:29,610 : INFO : EPOCH 12 - PROGRESS: at 69.26% examples, 208168 words/s, in_qsize 5, out_qsize 0
2020-08-12 17:35:30,616 : INFO : EPOCH 12 - PROGRESS: at 72.43% examples, 207993 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:35:31,671 : INFO : EPOCH 12 - PROGRESS: at 75.98% examples, 208082 words/s

2020-08-12 17:36:30,870 : INFO : EPOCH 14 - PROGRESS: at 60.87% examples, 209213 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:36:31,951 : INFO : EPOCH 14 - PROGRESS: at 64.23% examples, 209520 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:36:33,022 : INFO : EPOCH 14 - PROGRESS: at 67.60% examples, 211741 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:36:34,025 : INFO : EPOCH 14 - PROGRESS: at 70.92% examples, 212081 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:36:35,048 : INFO : EPOCH 14 - PROGRESS: at 74.17% examples, 212014 words/s, in_qsize 5, out_qsize 0
2020-08-12 17:36:36,087 : INFO : EPOCH 14 - PROGRESS: at 77.53% examples, 211678 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:36:37,115 : INFO : EPOCH 14 - PROGRESS: at 80.69% examples, 212876 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:36:38,152 : INFO : EPOCH 14 - PROGRESS: at 83.99% examples, 212426 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:36:39,166 : INFO : EPOCH 14 - PROGRESS: at 87.32% examples, 213429 words/s

2020-08-12 17:37:39,991 : INFO : EPOCH 16 - PROGRESS: at 69.37% examples, 208894 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:37:41,088 : INFO : EPOCH 16 - PROGRESS: at 72.66% examples, 208255 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:37:42,156 : INFO : EPOCH 16 - PROGRESS: at 76.22% examples, 208207 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:37:43,186 : INFO : EPOCH 16 - PROGRESS: at 79.76% examples, 210075 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:37:44,255 : INFO : EPOCH 16 - PROGRESS: at 83.10% examples, 210475 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:37:45,267 : INFO : EPOCH 16 - PROGRESS: at 86.52% examples, 211376 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:37:46,334 : INFO : EPOCH 16 - PROGRESS: at 89.91% examples, 212008 words/s, in_qsize 5, out_qsize 0
2020-08-12 17:37:47,396 : INFO : EPOCH 16 - PROGRESS: at 93.37% examples, 211787 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:37:48,434 : INFO : EPOCH 16 - PROGRESS: at 96.65% examples, 213022 words/s

2020-08-12 17:38:48,784 : INFO : EPOCH 18 - PROGRESS: at 71.56% examples, 213906 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:38:49,851 : INFO : EPOCH 18 - PROGRESS: at 74.87% examples, 213120 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:38:50,872 : INFO : EPOCH 18 - PROGRESS: at 78.00% examples, 212839 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:38:51,901 : INFO : EPOCH 18 - PROGRESS: at 81.38% examples, 214692 words/s, in_qsize 5, out_qsize 0
2020-08-12 17:38:52,956 : INFO : EPOCH 18 - PROGRESS: at 84.78% examples, 214035 words/s, in_qsize 5, out_qsize 0
2020-08-12 17:38:54,029 : INFO : EPOCH 18 - PROGRESS: at 88.03% examples, 214429 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:38:55,045 : INFO : EPOCH 18 - PROGRESS: at 91.48% examples, 215104 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:38:56,081 : INFO : EPOCH 18 - PROGRESS: at 94.10% examples, 213313 words/s, in_qsize 6, out_qsize 0
2020-08-12 17:38:57,095 : INFO : EPOCH 18 - PROGRESS: at 95.96% examples, 210841 words/s

KeyboardInterrupt: 